In [45]:
pip install feature-engine

In [119]:
import numpy as np 
import pandas as pd
import sklearn
import feature_engine
from feature_engine.encoding import OneHotEncoder as fe_OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import re
import joblib


In [47]:
df = pd.read_csv('/content/input_track_w_features_df.csv')

In [48]:
def explicit_binarizer(x):
  if x == True:
    return 1 
  else:
    return 0

In [49]:
def decade_function(x):
    if x in range(1900,1991):
        return '1900-1990'
    elif x in range(1991,1996):
        return '1991-1995'
    elif x in range(1996,2001):
        return '1996-2000'
    elif x in range(2001,2006):
        return '2001-2005'
    elif x == 2006:
        return '2006' 
    elif x == 2007:
        return '2007'
    elif x == 2008:
        return '2008'
    elif x == 2009:
        return '2009'
    elif x == 2010:
        return '2010'
    elif x == 2011:
        return '2011'
    elif x == 2012:
        return '2012'
    elif x == 2013:
        return '2013'
    elif x == 2014:
        return '2014'
    elif x == 2015:
        return '2015'
    elif x == 2016:
        return '2016'
    elif x == 2017:
        return '2017'
    elif x == 2018:
        return '2018'
    elif x == 2019:
        return '2019'
    elif x == 2020:
        return '2020'
    elif x == 2021:
        return '2021'

In [50]:
df.head()

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,explicit
0,0,0.596,0.824,4,-4.893,0,0.0361,0.0504,0.129,0.0586,0.786,135.171,4,1979,False


In [51]:
def wrangle(df):
  df.drop(columns='Unnamed: 0', inplace = True)
  df['explicit'] = df['explicit'].apply(explicit_binarizer)
  df['age'] = df['year'].apply(decade_function)
  loaded_ohe = joblib.load('/content/ohe.joblib')
  df = loaded_ohe.transform(df.fillna('Missing'))
  #feature scailing for float columns in df3
  loaded_scaler = joblib.load('/content/scaler.joblib')
  scale_cols = ['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'key', 'mode']
  scaled = df[scale_cols].reset_index(drop=True)
  scaler = loaded_scaler
  scaled_float_df = pd.DataFrame(scaler.transform(scaled), columns = scaled.columns)
  df.drop(columns = scale_cols, inplace = True)
  df = pd.concat([df, scaled_float_df], axis = 1)
  df.drop(columns='year', inplace = True)
  return df

In [52]:
df = wrangle(df)

In [68]:
# Load pickled model and recommendations lookup dataframe
knn_loader = joblib.load('/content/knn_model.joblib')
!unzip '/content/df_rec_lookup.zip'
df_rec_lookup = pd.read_csv('df_rec_lookup.csv')

Archive:  /content/df_rec_lookup.zip
replace df_rec_lookup.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: [y]
error:  invalid response [[y]]
replace df_rec_lookup.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: df_rec_lookup.csv       


In [118]:
# Query Using kneighbors 
__, neigh_index = knn_loader.kneighbors(df)

# Instantiate song list
song_list = []

for i in neigh_index[0][:]:
  #df_rec_lookup['artists'][i] = df_rec_lookup['artists'].apply(lambda x: "".join(x))
  
  song_list.append(f"{df_rec_lookup['name'][i]} by {df_rec_lookup['artists'][i]}")

for i in song_list:
  i = re.sub("[^a-zA-Z 0-9 \u00d6 , -]", '', i)
  print(i)


I Am the Storm by Blue Öyster Cult
Fourth Day of July by The Washington Squares
Texas by Charlie Daniels, The Charlie Daniels Band
Sing Baby Sing by The Stylistics
China Girl - 2018 Remaster by David Bowie
Razors Edge by Meat Loaf
